In [8]:
# Data Cleaning Ideas:

# One-hot encoding for Y/N columns
# Encode non-binary columns like furniture status
# removing duplicates---idk if we have any but we can find code to check


# Normalizing/scaling necessary for linear regression -- i dont rlly know what that means

# Visuals
# visuals-- still confused on where/how its needed but we can just use a boxplot or scatter plot. will be good to show outlier detection
# Boxplot, Scatter Plot: area and prices, heatmap
# we can use this to add to our presentation and discuss why or why not we choose to remove them: IQR/z-score

import pandas as pd
from sklearn.preprocessing import StandardScaler


house_df = pd.read_csv("C:\\Users\\AidaBah\\Desktop\\csci1070_course\\mycsclass\\csfinalproject\\Housing.csv")

house_df

def one_hot_encoding(df):
    yn_columns = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
    for col in yn_columns:
        df[col] = df[col].apply(lambda x: 1 if x == 'yes' else 0)
    return df

def encode_categorical(df):
    non_binary_cols = ['furnishingstatus']
    df = pd.get_dummies(df, columns=non_binary_cols, drop_first=True)
    return df

def remove_duplicates(df):
    df = df.drop_duplicates()
    return df

def normalize_data(df):
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return df_scaled

house_df = one_hot_encoding(house_df)
house_df = encode_categorical(house_df)
house_df = remove_duplicates(house_df)
house_df = normalize_data(house_df)

house_df.to_csv('cleaned_house.csv')
